# Demonstration
This is code to demonstrate how to work with the **Azure Synapse Studio** notebooks.
You can get more help at [Create, develop, and maintain Synapse notebooks in Azure Synapse Analytics](https://docs.microsoft.com/en-us/azure/synapse-analytics/spark/apache-spark-development-using-notebooks).

We will examine the following:

1. Markdown cells [Markdown for Jupyter notebooks cheatsheet](https://www.ibm.com/docs/en/watson-studio-local/1.2.3?topic=notebooks-markdown-jupyter-cheatsheet)
1. Working with the file magic commands
1. Using the mssparkutil class
1. Access file as dataset
1. Display options
    1. Show column information
    1. Show sample data
    1. Display in charts
1. Using multiple languages
1. Using SQL

We won't go deep into how to actually develop.  My goal is to help you understand some of the mechanics of working with the notebooks.

In [41]:
print("Hello world!")

## File magic commands
The first thing I like to do is to get my bearings in the file system.
There are enough places that I am going to mess up.  I need to make sure that the file 
is where I think that it is.

These are some of the commands that **_might_** be available in Azure Synapse Studio.  https://ipython.readthedocs.io/en/stable/interactive/magics.html.

Try %lsmagic to get a full list of the commands.

### lsmagic
This provides a list of the magic commands with a link to the ipython document mentioned above.

In [42]:
%lsmagic fs

### fs magic to get the filesystem information
This is where I start to learn about what is on the drive.

The only documentation that I have found for the %fs magic is the mssparkutils documentation for the fs class.  https://docs.microsoft.com/en-us/azure/synapse-analytics/spark/microsoft-spark-utilities?pivots=programming-language-python#file-system-utilities



In [43]:
%fs ls "abfss://loskisynapsefilesystem@loskisynapsedatalake.dfs.core.windows.net/census/PUMS/1-year/2017/Population/"


What is in a file?

`%fs head`

In [44]:
%fs head abfss://loskisynapsefilesystem@loskisynapsedatalake.dfs.core.windows.net/census/PUMS/1-year/2017/Population/psam_pusa.csv  

In [45]:
%fs head abfss://loskisynapsefilesystem@loskisynapsedatalake.dfs.core.windows.net/census/PUMS/1-year/2017/Population/psam_pusa.csv 1100

## mssparkutil

mssparkutil is a utility class that you can use in several scenarios.  You can get directories of files or you can change the configuration of the your spark session.  If you are needing to do anything advanced with the Spark environment, I would see what is availabe in mssparkutil.

https://docs.microsoft.com/en-us/azure/synapse-analytics/spark/microsoft-spark-utilities?pivots=programming-language-python


Prepare by importing the library

In [46]:
from notebookutils import mssparkutils

In [47]:
folder = "abfss://loskisynapsefilesystem@loskisynapsedatalake.dfs.core.windows.net/census/PUMS/1-year/2017/Population/"
files = mssparkutils.fs.ls(folder)
for file in files:
    print(file.name, file.isDir, file.isFile, file.path, file.size)

In [48]:
for file in [f for f in files if f.name.endswith(".csv")]:
    print(file.name, file.isDir, file.isFile, file.path, file.size)
    print(mssparkutils.fs.head(file.path, 2000))

## Access file as dataset
Once I know where the file is, I want to query the file.  I can use different languages and tools to do that.  I am most comfortable using Python.

We won't delve deep into working with Python.

Where I start is on the data tab.  

In [49]:
df = spark.read.load('abfss://loskisynapsefilesystem@loskisynapsedatalake.dfs.core.windows.net/census/PUMS/1-year/2017/Population/psam_pusa.csv', format='csv'
## If header exists uncomment line below
##, header=True
)
display(df.limit(10))

Let's add headers.

In [50]:
df = spark.read.load('abfss://loskisynapsefilesystem@loskisynapsedatalake.dfs.core.windows.net/census/PUMS/1-year/2017/Population/psam_pusa.csv', format='csv'
## If header exists uncomment line below
, header=True
)
display(df.limit(100))

Let's view the schema information.

In [ ]:
df.printSchema()

In [ ]:
df = spark.read.load('abfss://loskisynapsefilesystem@loskisynapsedatalake.dfs.core.windows.net/census/PUMS/1-year/2017/Population/psam_pusa.csv', format='csv'
## If header exists uncomment line below
, header=True
)
df.printSchema()

The schema is entirely strings for all columns. 

Notice

Get a more detailed schema

In [55]:
df = spark.read.load('abfss://loskisynapsefilesystem@loskisynapsedatalake.dfs.core.windows.net/census/PUMS/1-year/2017/Population/psam_pusa.csv', format='csv'
## If header exists uncomment line below
, header=True
, inferSchema=True
)
display(df.limit(100))

In [56]:
df.printSchema()

Now we are going restrict the number of columns to make it easier to work with the data

In [61]:
from pyspark.sql import functions as func

dfHisp = df.filter((df.AGEP < 65) & (df.HISP != 1 )).groupBy("HICOV").agg(func.sum("PWGTP").alias("TotalPopulation"))


In [59]:
display(dfHisp)

## Use a different language
You can open a dataframe using one language and manipulate it using a different language

First you need to create a temporary table 

In [62]:
dfHisp.createOrReplaceTempView("hispanicInsurance")

Run SQL

In [63]:
%%sql
SELECT * FROM hispanicInsurance

Use C#

In [66]:
%%csharp

DataFrame df = spark.Sql("SELECT * FROM hispanicInsurance");

df.Show();

## Snippets
Type the word `Snippet` in a code cell and hit control-space. It will list some options for code snippets.  I was able to get this to work in Chrome but not Edge.

In [65]:
import matplotlib.pyplot as plt
import numpy as np

# Use numpy to generate a bunch of random data in a bell curve around 5.
n = 5 + np.random.randn(1000)

m = [m for m in range(len(n))]
plt.bar(m, n)
plt.title("Raw Data")
plt.show()

plt.hist(n, bins=20)
plt.title("Histogram")
plt.show()

plt.hist(n, cumulative=True, bins=20)
plt.title("Cumulative Histogram")
plt.show()

In [ ]:
from pyspark.sql import SparkSession 
from pyspark.sql.types import * 

# Primary storage info 
account_name = 'Your primary storage account name' # fill in your primary account name 
container_name = 'Your container name' # fill in your container name 
relative_path = 'Your relative path' # fill in your relative folder path 

adls_path = 'abfss://%s@%s.dfs.core.windows.net/%s' % (container_name, account_name, relative_path) 
print('Primary storage account path: ' + adls_path) 

# Read a csv file 
csv_path = adls_path + 'Your file name ' 
df_csv = spark.read.csv(csv_path, header = 'true') 

# Read a parquet file 
parquet_path = adls_path + ' Your file name ' 
df_parquet = spark.read.parquet(parquet_path) 

# Read a json file 
json_path = adls_path + 'Your file name ' 
df_json = spark.read.json(json_path) 